In [ ]:
import pandas as pd
import dask.dataframe as dd
import praw
from dask.diagnostics import ProgressBar
from tqdm import tqdm
import hashlib
from PIL import Image
import requests
from io import BytesIO
from shared_code.utility.schemas.pyarrow_schema import schema
from shared_code.utility.spark.set_environ import set_azure_env
import os
import torch
from datetime import datetime
import pandas as pd
from pmaw import PushshiftAPI

set_azure_env()

from shared_code.utility.storage.azure_file_storage import AzureFileStorageAdapter
from shared_code.utility.scripts.blip_caption import BlipCaption

pbar = ProgressBar()
pbar.register()

tqdm.pandas()
tqdm.pandas(desc="global")

from tqdm.dask import TqdmCallback
cb = TqdmCallback(desc="global")
cb.register()

file_system = AzureFileStorageAdapter('data').get_file_storage()

In [ ]:
def fetch_image(x: object) -> object:
	try:
		url = x['original_url']
		subreddit = x['subreddit']
		response = requests.get(url)
		md5 = hashlib.md5(response.content).hexdigest()
		os.makedirs(f"D:\\data\\images\\{subreddit}", exist_ok=True)
		out_path = f"D:\\data\\images\\{subreddit}\\{md5}.jpg"

		if os.path.exists(out_path):
			return out_path

		if md5 in list(extant_data['hash']):
			return out_path

		if md5 != "f17b01901c752c1bb04928131d1661af" or md5 != "d835884373f4d6c8f24742ceabe74946":
			raw_image = Image.open(BytesIO(response.content))
			raw_image.save(out_path)
			raw_image.close()
			return out_path
		else:
			return ""
	except Exception as e:
		print(e)
		return ""

In [ ]:
def make_thumbnail(x: object) -> str:
	image_path = x['path']
	md5 = x['hash']
	subreddit = x['subreddit']
	os.makedirs(f"D:\\data\\images\\{subreddit}\\thumbnail", exist_ok=True)
	out_path = f"D:\\data\\images\\{subreddit}\\thumbnail\\{md5}.jpg"
	if not os.path.exists(image_path):
		return ""
	if os.path.exists(out_path):
		return out_path
	img = Image.open(image_path)
	try:
		copied_image = img.copy()
		result_copy = copied_image.resize((512, 512))
		result_copy.save(out_path)
		result_copy.close()
		img.close()
		return out_path
	except Exception as e:
		print(e)
		return ""
	finally:
		img.close()

In [ ]:
def exists(x: object) -> bool:
	try:
		image_path = os.path.exists(x['path'])
		thumbnail_path = os.path.exists(x['thumbnail_path'])
		return image_path and thumbnail_path
	except Exception as e:
		print(e)
		return False

In [ ]:
def split_hash(x: object) -> str:
	try:
		name = x['image_name']
		return name.split('.')[0]
	except Exception as e:
		print(e)
		return ""

In [ ]:
def caption_image(x: object, blip_captioning: BlipCaption) -> str:
	try:
		path = x['path']
		exists_image = bool(x['exists'])
		resulting_caption = x['caption']

		if not os.path.exists(path):
			return ""

		if not exists_image:
			return ""

		if len(resulting_caption) > 5:
			return resulting_caption

		resulting_caption = blip_captioning.caption_image(path)
		return resulting_caption

	except Exception as e:
		return ""
	finally:
		pass

In [ ]:
def get_image_name(x: object) -> str:
	try:
		path = x['path']
		if path == "":
			return ""
		return os.path.basename(path)
	except Exception as e:
		print(e)
		return ""